In [11]:
import networkx as nx
import dill as pickle
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import scipy.optimize as opt
import copy
from mpl_toolkits.basemap import Basemap

In [12]:
# Read pickled graphs graph_objects/G_multi_directed.pickle, graph_objects/G_multi_undirected.pickle, graph_objects/G_simple_directed.pickle, graph_objects/G_simple_undirected.pickle
with open('graph_objects/G_multi_directed_iggielgn.pickle', 'rb') as f:
    G_multi_directed = pickle.load(f)

with open('graph_objects/G_multi_undirected_iggielgn.pickle', 'rb') as f:
    G_multi_undirected = pickle.load(f)

with open('graph_objects/G_simple_directed_iggielgn.pickle', 'rb') as f:
    G_simple_directed = pickle.load(f)

with open('graph_objects/G_simple_undirected_iggielgn.pickle', 'rb') as f:
    G_simple_undirected = pickle.load(f)

In [13]:
import random

# Assuming G_multi_directed is your NetworkX graph
random_node = random.choice(list(G_multi_directed.nodes))
random_edge = random.choice(list(G_multi_directed.edges))

print('Attributes of a random node:')
print(G_multi_directed.nodes[random_node])

print('Attributes of a random edge:')
print(G_multi_directed.edges[random_edge])

Attributes of a random node:
{'id': 'N_101_L_LMGN', 'name': 'N_101_L_LMGN', 'source_id': "['IT_1707']", 'node_id': "['N_101_L_LMGN']", 'lat': 41.3134, 'long': 14.6161, 'country_code': 'IT', 'comment': nan, 'tags': '{}', 'nuts_id_1': 'ITF', 'nuts_id_2': 'ITF3', 'nuts_id_3': 'ITF32', 'uncertainty_nuts_id_1': 0, 'uncertainty_nuts_id_2': 0, 'uncertainty_nuts_id_3': 0, 'method_nuts_id_1': 'M_Consumers.add_Nuts_Num()', 'method_nuts_id_2': 'M_Consumers.add_Nuts_Num()', 'method_nuts_id_3': 'M_Consumers.add_Nuts_Num()', 'type': 'node', 'flow_type': 'transit'}
Attributes of a random edge:
{'id': 'PLM_2090_Seg_0_Seg_0_Seg_0', 'name': 'PLM_2090_Seg_0_Seg_0_Seg_0', 'source_id': "['EntsoG_Map_51220', 'INET_PL_682']", 'comment': nan, 'tags': '{}', 'diameter_mm': 900.0, 'end_year': None, 'is_H_gas': 1, 'is_bothDirection': 1, 'lat_mean': 47.45062, 'length_km': 12.635303, 'long_mean': 7.519408, 'max_cap_M_m3_per_d': 19.618598434835253, 'max_pressure_bar': 67.02092561304427, 'nuts_id_1': "['CH0', 'FRF']"

In [14]:
# Create a dictionary. The keys are the country_codes and the values are 1) the number of nodes in the country 2) the number of to_edges in the country 3) the number of from_edges in the country 4) the number of total edges in the country

country_dict = {}
for node in G_multi_directed.nodes:
    country_code = G_multi_directed.nodes[node]['country_code']
    if country_code in country_dict:
        country_dict[country_code][0] += 1
    else:
        country_dict[country_code] = [1, 0, 0, 0]

In [15]:
# Print the country_dict
for country in country_dict:
    print(country, country_dict[country])

PL [198, 0, 0, 0]
IT [373, 0, 0, 0]
AL [2, 0, 0, 0]
SK [18, 0, 0, 0]
GB [178, 0, 0, 0]
BY [87, 0, 0, 0]
GR [33, 0, 0, 0]
EE [24, 0, 0, 0]
TR [180, 0, 0, 0]
UA [197, 0, 0, 0]
RU [372, 0, 0, 0]
XX [513, 0, 0, 0]
NL [102, 0, 0, 0]
ES [210, 0, 0, 0]
BE [70, 0, 0, 0]
FR [622, 0, 0, 0]
HR [36, 0, 0, 0]
CZ [41, 0, 0, 0]
SI [23, 0, 0, 0]
HU [99, 0, 0, 0]
RS [42, 0, 0, 0]
AT [51, 0, 0, 0]
SE [18, 0, 0, 0]
RO [162, 0, 0, 0]
IE [52, 0, 0, 0]
MD [19, 0, 0, 0]
FI [40, 0, 0, 0]
BG [75, 0, 0, 0]
LT [35, 0, 0, 0]
DK [21, 0, 0, 0]
CH [28, 0, 0, 0]
PT [29, 0, 0, 0]
LV [20, 0, 0, 0]
AZ [9, 0, 0, 0]
GE [35, 0, 0, 0]
AM [9, 0, 0, 0]
LU [5, 0, 0, 0]
MK [4, 0, 0, 0]
DE [631, 0, 0, 0]
NO [7, 0, 0, 0]
BA [2, 0, 0, 0]
EL [2, 0, 0, 0]
MT [1, 0, 0, 0]
